In [ ]:
import tensorflow as tf
from deeplabv3plus.model.deeplabv3_plus import DeeplabV3Plus

In [ ]:
model = DeeplabV3Plus(backbone='resnet50',num_classes=20)
input_shape = (1,256,256,3)
input_tensor = tf.random.normal(input_shape)
result = model(input_tensor)

In [ ]:
print(result)

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath="checkpoints/deeplabv3-plus-human-parsing-resnet-50-backbone_12"

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
                    learning_rate=0.001
                ),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy']
            )


In [ ]:
from glob import glob
from deeplabv3plus.inference import infer, read_image
from deeplabv3plus.utils import plot_samples_matplotlib

In [ ]:
train_images = glob('./dataset/instance-level_human_parsing/instance-level_human_parsing/Training/Images/*')
val_images = glob('./dataset/instance-level_human_parsing/instance-level_human_parsing/Validation/Images/*')
test_images = glob('./dataset/instance-level_human_parsing/instance-level_human_parsing/Testing/Images/*')

In [ ]:
def plot_predictions(images_list, size):
    for image_file in images_list:
        image_tensor = read_image(image_file, size)
        prediction = infer(
            image_tensor=image_tensor,
            model_file=model
        )
        plot_samples_matplotlib(
            [image_tensor, prediction], figsize=(10, 6)
        )


In [ ]:
plot_predictions(train_images[:4], (256, 256))

In [ ]:
model.save("save/model")

In [ ]:
!pip install onnxruntime tf2onnx blobconverter
!python -m tf2onnx.convert --saved-model "save/model" --output model.onnx --opset 12

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse("save/model/model.xml")
root = tree.getroot()
data = root.find('.//layer[@name="strided_slice_10/extend_end_const1245431561"]/data')
data.set("element_type", "i32")

tree.write("save/model/export_model.xml")

In [ ]:
blob_export_dir = "export_dir"

binfile = "save/model/export_model.bin"
xmlfile = "save/model/export_model.xml"